In [1]:
import pandas as pd
import json

def the_odds_read_json(path) -> pd.DataFrame:
    # Reads in our collected json data from the odds api

    try:
        with open(path) as f:
            raw_json = json.load(f)

    except (FileNotFoundError, PermissionError) as e:
        print(f'Error accessing or modifying the file: {e}')
        return None
    except Exception as e:
        print(f'Error opening data: {e}')
        return None

    df = pd.json_normalize(raw_json, record_path=['bookmakers', 'markets', 'outcomes'],
                           meta=['id', 'sport_key', 'commence_time', 'sport_title', ['bookmakers', 'title'],
                                 ['bookmakers', 'key'], ['bookmakers', 'last_update'],['bookmakers', 'markets', 'key']])
    return df

#df = the_odds_read_json('../src/saved_data/apis/the_odds.json')

In [5]:
# returns a list of ArbitrageOpportunities

df = the_odds_read_json('../src/saved_data/apis/the_odds.json')


# Group by id and outcome name
df = df.loc[df[df['bookmakers.markets.key'] == 'h2h'].groupby(['name', 'id'])['price'].idxmax()]

# Sum one over the odds
df['one-on-price'] = 1.0 /df['price']

summed_odds = df.groupby(['id']).sum(numeric_only=True)

arbi_opportunities = summed_odds[summed_odds['one-on-price'] < 1]

# Combine arbi_opportunities into the same df
combined = df.join(summed_odds, on='id', how='inner', lsuffix='_df', rsuffix='_arbi')

# Iterate through each event and create an arbitrage opportunity object
ids = df['id']

for event_id in ids:
    relevant_info = combined[combined['id'] == event_id]
    #print(relevant_info)
    print(relevant_info)
    break

                 name  price_df  point_df                                id  \
87  Baltimore Orioles      2.00       NaN  1c7c1b9957b53cbc52aeda8838b9f3f6   
88  Toronto Blue Jays      1.95       NaN  1c7c1b9957b53cbc52aeda8838b9f3f6   

       sport_key         commence_time sport_title bookmakers.title  \
87  baseball_mlb  2023-06-14T23:05:00Z         MLB          Betfair   
88  baseball_mlb  2023-06-14T23:05:00Z         MLB          Betfair   

   bookmakers.key bookmakers.last_update bookmakers.markets.key  \
87        betfair   2023-06-14T06:43:18Z                    h2h   
88        betfair   2023-06-14T06:43:18Z                    h2h   

    one-on-price_df  price_arbi  point_arbi  one-on-price_arbi  
87         0.500000        3.95         0.0           1.012821  
88         0.512821        3.95         0.0           1.012821  


In [14]:
# Calculating the betting ratio
# FOR EQUAL profit based on the outcome
test = combined[combined['id'] == '8aca736723479efbc20fa5981d613472']
print(test)

def even_profit(df: pd.DataFrame) -> dict:
    # Iterate through each possible outcome (win, draw, lose) and then follow the equations
    # s1*o1 = s2*o2 = s3*o3
    # which means we can solve it for each one with

    num_outcomes = df.shape[0]
    ratios = {}

    for i in range(num_outcomes):
        divisor = 0

        for j in range(num_outcomes):
            divisor = divisor + (df.iloc[i]['price_df']/df.iloc[j]['price_df'])
        
        ratios[df.iloc[i]['name']] = 1/divisor
    
    return ratios

even_profit(test)


           name  price_df  point_df                                id  \
59      Croatia      4.30       NaN  8aca736723479efbc20fa5981d613472   
61         Draw      3.55       NaN  8aca736723479efbc20fa5981d613472   
57  Netherlands      2.04       NaN  8aca736723479efbc20fa5981d613472   

                     sport_key         commence_time          sport_title  \
59  soccer_uefa_nations_league  2023-06-14T18:45:00Z  UEFA Nations League   
61  soccer_uefa_nations_league  2023-06-14T18:45:00Z  UEFA Nations League   
57  soccer_uefa_nations_league  2023-06-14T18:45:00Z  UEFA Nations League   

   bookmakers.title bookmakers.key bookmakers.last_update  \
59          Betfair        betfair   2023-06-14T06:42:18Z   
61          Betfair        betfair   2023-06-14T06:42:18Z   
57           Unibet         unibet   2023-06-14T06:43:39Z   

   bookmakers.markets.key  one-on-price_df  price_arbi  point_arbi  \
59                    h2h         0.232558        9.89         0.0   
61           

{'Croatia': 0.23152914095719168,
 'Draw': 0.2804437482016689,
 'Netherlands': 0.48802711084113937}

In [22]:
def opportunity_to_string(df: pd.DataFrame) -> str:
    '''
    Converts the gross Dataframe into a nice readable string for our message library
    '''
    arbi_string = ''
    for i in range(df.shape[0]):
        name = df.iloc[i]['name']
        arbi_string = arbi_string + 'For OUTCOME: {} place bet proportion {} for EQUAL and {} if FAVOURED\n'.format(name, even_profit(df)[name], 'TODO')
    return arbi_string
print(opportunity_to_string(test))

For OUTCOME: Croatia place bet proportion 0.23152914095719168 for EQUAL and TODO if FAVOURED
For OUTCOME: Draw place bet proportion 0.2804437482016689 for EQUAL and TODO if FAVOURED
For OUTCOME: Netherlands place bet proportion 0.48802711084113937 for EQUAL and TODO if FAVOURED

